<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## LSTM Traductor
Ejemplo basado en [LINK](https://stackabuse.com/python-for-nlp-neural-machine-translation-with-seq2seq-in-keras/)

In [1]:
import re

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense
from tensorflow.keras.layers import Flatten, LSTM, SimpleRNN
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input
from keras.utils import plot_model

In [2]:
# Descargar la carpeta de dataset

import os
if os.access('spa-eng', os.F_OK) is False:
    if os.access('spa-eng.zip', os.F_OK) is False:
        !curl -L -o 'spa-eng.zip' 'http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip'
    !unzip -q spa-eng.zip
else:
    print("El dataset ya se encuentra descargado")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2576k  100 2576k    0     0  1775k      0  0:00:01  0:00:01 --:--:-- 1775k


In [3]:
import tensorflow as tf
import numpy as np

# Ruta al archivo
text_file = "./spa-eng/spa.txt"

# 1. Crear el dataset base leyendo directamente del archivo
# Esto NO carga el archivo en memoria, crea un puntero de lectura.
raw_dataset = tf.data.TextLineDataset(text_file)

# 2. Función para procesar cada línea (graph mode)
def process_text_line(line):
    # Separar por tabulador usando operaciones de TensorFlow
    parts = tf.strings.split(line, sep='\t')

    # Protección: asegurarnos de que la línea tenga las 2 partes (eng, spa)
    # Si la línea está rota, TF podría fallar, pero spa.txt suele estar limpio.
    # En un caso real, haríamos un filter() antes.
    input_text = parts[0]  # Inglés
    output_text = parts[1] # Español

    # Agregar tokens especiales
    # Nota: tf.strings.join es más eficiente que usar +
    decoder_input = tf.strings.join(['<sos> ', output_text])
    decoder_output = tf.strings.join([output_text, ' <eos>'])

    return input_text, decoder_input, decoder_output

# 3. Aplicar las transformaciones
# - Filter: filtramos líneas vacías o sin tabulador si fuera necesario (opcional)
# - Map: aplicamos la función de limpieza
dataset = raw_dataset.map(process_text_line)

# 4. Configurar el pipeline de entrenamiento
BATCH_SIZE = 64
BUFFER_SIZE = 20000 # Cuántas oraciones mantener en memoria para el shuffle

# - Shuffle: Mezcla usando un buffer (no mezcla todo el disco, solo lo que cabe en buffer)
# - Batch: Agrupa en lotes
# - Prefetch: Carga el siguiente lote en CPU mientras la GPU entrena el actual (paralelismo)
train_dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# --- VERIFICACIÓN ---
# Tomamos un batch para ver que funcione
print("Ejemplo de un batch procesado:")
for eng, spa_in, spa_out in train_dataset.take(1):
    print(f"Input (Eng): {eng[0].numpy().decode('utf-8')}")
    print(f"Dec In (Spa): {spa_in[0].numpy().decode('utf-8')}")
    print(f"Dec Out (Spa): {spa_out[0].numpy().decode('utf-8')}")
    print(f"Shape del batch: {eng.shape}")

Ejemplo de un batch procesado:
Input (Eng): Just hang in there.
Dec In (Spa): <sos> Solo espera ahí.
Dec Out (Spa): Solo espera ahí. <eos>
Shape del batch: (64,)


### 2 - Preprocesamiento

In [4]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Configuración
MAX_VOCAB_SIZE = 20000  # Subimos un poco ya que tenemos FastText
MAX_LEN_INPUT = 32
MAX_LEN_OUTPUT = 36

# --- 1. TOKENIZACIÓN (Inglés - Encoder) ---
input_tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE)
input_tokenizer.fit_on_texts(input_sentences) # input_sentences viene de tu carga anterior
input_integer_seq = input_tokenizer.texts_to_sequences(input_sentences)

word2idx_inputs = input_tokenizer.word_index
print(f"Vocabulario Entrada (Inglés): {len(word2idx_inputs)}")

# --- 2. TOKENIZACIÓN (Español - Decoder) ---
# Filters: Sacamos < y > para no romper <sos> y <eos>
output_tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, filters='!"#$%&()*+,-./:;=¿?@[\\]^_`{|}~\t\n')
output_tokenizer.fit_on_texts(output_sentences + ["<sos>", "<eos>"])

output_integer_seq = output_tokenizer.texts_to_sequences(output_sentences) # Target (sin sos)
output_input_integer_seq = output_tokenizer.texts_to_sequences(output_sentences_inputs) # Input (con sos)

word2idx_outputs = output_tokenizer.word_index
num_words_output = min(len(word2idx_outputs) + 1, MAX_VOCAB_SIZE)
print(f"Vocabulario Salida (Español): {len(word2idx_outputs)}")

# --- 3. PADDING ---
encoder_input_sequences = pad_sequences(input_integer_seq, maxlen=MAX_LEN_INPUT, padding='post')
decoder_input_sequences = pad_sequences(output_input_integer_seq, maxlen=MAX_LEN_OUTPUT, padding='post')
# NOTA: Usamos los enteros directamente como targets. ¡Nada de to_categorical!
decoder_targets = pad_sequences(output_integer_seq, maxlen=MAX_LEN_OUTPUT, padding='post')

print(f"Encoder Input Shape: {encoder_input_sequences.shape}")
print(f"Decoder Target Shape: {decoder_targets.shape}") # (30000, 36) -> Muy liviano en RAM

NameError: name 'input_sentences' is not defined

### 3 - Preparar los embeddings

In [21]:
# Descargar los embeddings desde un google drive (es la forma más rápida)
# NOTA: No hay garantía de que estos links perduren, en caso de que no estén
# disponibles descargar de la página oficial como se explica en el siguiente bloque de código
import os
import gdown
if os.access('gloveembedding.pkl', os.F_OK) is False:
    url = 'https://drive.google.com/uc?id=1KY6avD5I1eI2dxQzMkR3WExwKwRq2g94&export=download'
    output = 'gloveembedding.pkl'
    gdown.download(url, output, quiet=False)
else:
    print("Los embeddings gloveembedding.pkl ya están descargados")

Los embeddings gloveembedding.pkl ya están descargados


In [22]:
import logging
import os
from pathlib import Path
from io import StringIO
import pickle

class WordsEmbeddings(object):
    logger = logging.getLogger(__name__)

    def __init__(self):
        # load the embeddings
        words_embedding_pkl = Path(self.PKL_PATH)
        if not words_embedding_pkl.is_file():
            words_embedding_txt = Path(self.WORD_TO_VEC_MODEL_TXT_PATH)
            assert words_embedding_txt.is_file(), 'Words embedding not available'
            embeddings = self.convert_model_to_pickle()
        else:
            embeddings = self.load_model_from_pickle()
        self.embeddings = embeddings
        # build the vocabulary hashmap
        index = np.arange(self.embeddings.shape[0])
        # Dicctionarios para traducir de embedding a IDX de la palabra
        self.word2idx = dict(zip(self.embeddings['word'], index))
        self.idx2word = dict(zip(index, self.embeddings['word']))

    def get_words_embeddings(self, words):
        words_idxs = self.words2idxs(words)
        return self.embeddings[words_idxs]['embedding']

    def words2idxs(self, words):
        return np.array([self.word2idx.get(word, -1) for word in words])

    def idxs2words(self, idxs):
        return np.array([self.idx2word.get(idx, '-1') for idx in idxs])

    def load_model_from_pickle(self):
        self.logger.debug(
            'loading words embeddings from pickle {}'.format(
                self.PKL_PATH
            )
        )
        max_bytes = 2**28 - 1 # 256MB
        bytes_in = bytearray(0)
        input_size = os.path.getsize(self.PKL_PATH)
        with open(self.PKL_PATH, 'rb') as f_in:
            for _ in range(0, input_size, max_bytes):
                bytes_in += f_in.read(max_bytes)
        embeddings = pickle.loads(bytes_in)
        self.logger.debug('words embeddings loaded')
        return embeddings

    def convert_model_to_pickle(self):
        # create a numpy strctured array:
        # word     embedding
        # U50      np.float32[]
        # word_1   a, b, c
        # word_2   d, e, f
        # ...
        # word_n   g, h, i
        self.logger.debug(
            'converting and loading words embeddings from text file {}'.format(
                self.WORD_TO_VEC_MODEL_TXT_PATH
            )
        )
        structure = [('word', np.dtype('U' + str(self.WORD_MAX_SIZE))),
                     ('embedding', np.float32, (self.N_FEATURES,))]
        structure = np.dtype(structure)
        # load numpy array from disk using a generator
        with open(self.WORD_TO_VEC_MODEL_TXT_PATH, encoding="utf8") as words_embeddings_txt:
            embeddings_gen = (
                (line.split()[0], line.split()[1:]) for line in words_embeddings_txt
                if len(line.split()[1:]) == self.N_FEATURES
            )
            embeddings = np.fromiter(embeddings_gen, structure)
        # add a null embedding
        null_embedding = np.array(
            [('null_embedding', np.zeros((self.N_FEATURES,), dtype=np.float32))],
            dtype=structure
        )
        embeddings = np.concatenate([embeddings, null_embedding])
        # dump numpy array to disk using pickle
        max_bytes = 2**28 - 1 # # 256MB
        bytes_out = pickle.dumps(embeddings, protocol=pickle.HIGHEST_PROTOCOL)
        with open(self.PKL_PATH, 'wb') as f_out:
            for idx in range(0, len(bytes_out), max_bytes):
                f_out.write(bytes_out[idx:idx+max_bytes])
        self.logger.debug('words embeddings loaded')
        return embeddings


class GloveEmbeddings(WordsEmbeddings):
    WORD_TO_VEC_MODEL_TXT_PATH = 'glove.twitter.27B.50d.txt'
    PKL_PATH = 'gloveembedding.pkl'
    N_FEATURES = 50
    WORD_MAX_SIZE = 60

class FasttextEmbeddings(WordsEmbeddings):
    WORD_TO_VEC_MODEL_TXT_PATH = 'cc.en.300.vec'
    PKL_PATH = 'fasttext.pkl'
    N_FEATURES = 300
    WORD_MAX_SIZE = 60

In [23]:
# Por una cuestion de RAM se utilizarán los embeddings de Glove de dimension 50
model_embeddings = GloveEmbeddings()

In [28]:
# Crear la Embedding matrix de las secuencias
# en inglés

print('preparing embedding matrix...')
embed_dim = model_embeddings.N_FEATURES
words_not_found = []

# word_index provieen del tokenizer

nb_words = min(MAX_VOCAB_SIZE, len(word2idx_inputs)) # vocab_size
embedding_matrix = np.zeros((nb_words, embed_dim))
for word, i in word2idx_inputs.items():
    if i >= nb_words:
        continue
    embedding_vector = model_embeddings.get_words_embeddings(word)[0]
    if (embedding_vector is not None) and len(embedding_vector) > 0:

        embedding_matrix[i] = embedding_vector
    else:
        # words not found in embedding index will be all-zeros.
        words_not_found.append(word)

print('number of null word embeddings:', np.sum(np.sum(embedding_matrix**2, axis=1) == 0))

preparing embedding matrix...
number of null word embeddings: 98


In [29]:
# Dimensión de los embeddings de la secuencia en inglés
embedding_matrix.shape

(8121, 50)

### 4 - Entrenar el modelo

In [30]:
max_input_len

32

In [32]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
import tensorflow as tf

# Asumo que estas variables ya las tenés definidas arriba:
# n_units, max_input_len, nb_words, embed_dim, embedding_matrix,
# num_words_output, max_out_len

n_units = 128

# --- ENCODER ---
encoder_inputs = Input(shape=(max_input_len,), name='Encoder_Input')

encoder_embedding_layer = Embedding(
    input_dim=nb_words,          # Tamaño del vocabulario de entrada
    output_dim=embed_dim,        # Dimensión del vector denso (ej. 50 para GloVe)
    weights=[embedding_matrix],  # Cargar pesos pre-entrenados
    trainable=False,             # Congelar para no re-entrenar GloVe
    name='Encoder_Embedding'
)

encoder_inputs_x = encoder_embedding_layer(encoder_inputs)

# LSTM Encoder
encoder = LSTM(n_units, return_state=True, name='Encoder_LSTM')
encoder_outputs, state_h, state_c = encoder(encoder_inputs_x)

# Guardamos los estados para pasarlos al decoder (Context Vector)
encoder_states = [state_h, state_c]

# --- DECODER ---
decoder_inputs = Input(shape=(max_out_len,), name='Decoder_Input')

decoder_embedding_layer = Embedding(
    input_dim=num_words_output,
    output_dim=n_units,
    name='Decoder_Embedding'
)
decoder_inputs_x = decoder_embedding_layer(decoder_inputs)

# LSTM Decoder
# return_sequences=True es vital aquí porque queremos la predicción palabra por palabra
decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True, name='Decoder_LSTM')

# Inicializamos el decoder con los estados del encoder
decoder_outputs, _, _ = decoder_lstm(decoder_inputs_x, initial_state=encoder_states)

# --- SALIDA ---
decoder_dense = Dense(num_words_output, activation='softmax', name='Output_Layer')
decoder_outputs = decoder_dense(decoder_outputs)

# --- MODELO ---
model = Model([encoder_inputs, decoder_inputs], decoder_outputs, name='Seq2Seq_Translator')

model.compile(loss='categorical_crossentropy', optimizer="Adam", metrics=['accuracy'])

model.summary()

Model: "Seq2Seq_Translator"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Encoder_Input       │ (None, 32)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Decoder_Input       │ (None, 36)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Encoder_Embedding   │ (None, 32, 50)    │    406,050 │ Encoder_Input[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Decoder_Embedding   │ (None, 36, 128)   │  1,280,000 │ Decoder_Input[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Encoder_LSTM (LSTM) │ [(None, 128),     │     91,648 │ Encoder_Embeddin… │
│                     │ (None, 128),      │            │                   │
│                     │ (None, 128)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Decoder_LSTM (LSTM) │ [(None, 36, 128), │    131,584 │ Decoder_Embeddin… │
│                     │ (None, 128),      │            │ Encoder_LSTM[0][… │
│                     │ (None, 128)]      │            │ Encoder_LSTM[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Output_Layer        │ (None, 36, 10000) │  1,290,000 │ Decoder_LSTM[0][… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,199,282 (12.20 MB)

 Trainable params: 2,793,232 (10.66 MB)

 Non-trainable params: 406,050 (1.55 MB)

In [33]:
encoder_model = Model(encoder_inputs, encoder_states)

# define inference decoder
decoder_state_input_h = Input(shape=(n_units,))
decoder_state_input_c = Input(shape=(n_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# En cada predicción habrá una sola palabra de entrada al decoder,
# que es la realimentación de la palabra anterior
# por lo que hay que modificar el input shape de la layer de Embedding
decoder_inputs_single = Input(shape=(1,))
decoder_inputs_single_x = decoder_embedding_layer(decoder_inputs_single)

decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs_single_x, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs_single] + decoder_states_inputs, [decoder_outputs] + decoder_states)



In [ ]:
hist = model.fit(
    [encoder_input_sequences, decoder_input_sequences],
    decoder_targets,
    epochs=100,
    validation_split=0.2)

Epoch 1/100
750/750 ━━━━━━━━━━━━━━━━━━━━ 48s 58ms/step - accuracy: 0.8140 - loss: 2.1922 - val_accuracy: 0.8449 - val_loss: 1.0518
Epoch 2/100
750/750 ━━━━━━━━━━━━━━━━━━━━ 31s 42ms/step - accuracy: 0.8439 - loss: 1.0570 - val_accuracy: 0.8527 - val_loss: 0.9634
Epoch 3/100
750/750 ━━━━━━━━━━━━━━━━━━━━ 31s 42ms/step - accuracy: 0.8527 - loss: 0.9536 - val_accuracy: 0.8584 - val_loss: 0.8957
Epoch 4/100
750/750 ━━━━━━━━━━━━━━━━━━━━ 31s 42ms/step - accuracy: 0.8590 - loss: 0.8772 - val_accuracy: 0.8623 - val_loss: 0.8553
Epoch 5/100
750/750 ━━━━━━━━━━━━━━━━━━━━ 31s 42ms/step - accuracy: 0.8632 - loss: 0.8237 - val_accuracy: 0.8654 - val_loss: 0.8229
Epoch 6/100
750/750 ━━━━━━━━━━━━━━━━━━━━ 31s 42ms/step - accuracy: 0.8667 - loss: 0.7782 - val_accuracy: 0.8673 - val_loss: 0.8008
Epoch 7/100
750/750 ━━━━━━━━━━━━━━━━━━━━ 31s 42ms/step - accuracy: 0.8711 - loss: 0.7312 - val_accuracy: 0.8689 - val_loss: 0.7841
Epoch 8/100
750/750 ━━━━━━━━━━━━━━━━━━━━ 31s 42ms/step - accuracy: 0.8738 - loss: 0

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Entrenamiento
epoch_count = range(1, len(hist.history['accuracy']) + 1)
sns.lineplot(x=epoch_count,  y=hist.history['accuracy'], label='train')
sns.lineplot(x=epoch_count,  y=hist.history['val_accuracy'], label='valid')
plt.show()

### 5 - Inferencia

In [ ]:
'''
Step 1:
A deal is a deal -> Encoder -> enc(h1,c1)

enc(h1,c1) + <sos> -> Decoder -> Un + dec(h1,c1)

step 2:
dec(h1,c1) + Un -> Decoder -> trato + dec(h2,c2)

step 3:
dec(h2,c2) + trato -> Decoder -> es + dec(h3,c3)

step 4:
dec(h3,c3) + es -> Decoder -> un + dec(h4,c4)

step 5:
dec(h4,c4) + un -> Decoder -> trato + dec(h5,c5)

step 6:
dec(h5,c5) + trato. -> Decoder -> <eos> + dec(h6,c6)
'''

In [ ]:
# Armar los conversores de índice a palabra:
idx2word_input = {v:k for k, v in word2idx_inputs.items()}
idx2word_target = {v:k for k, v in word2idx_outputs.items()}

In [ ]:
def translate_sentence(input_seq):
    # Se transforma la sequencia de entrada a los estados "h" y "c" de la LSTM
    # para enviar la primera vez al decoder
    states_value = encoder_model.predict(input_seq)

    # Se inicializa la secuencia de entrada al decoder como "<sos>"
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = word2idx_outputs['<sos>']

    # Se obtiene el índice que finaliza la inferencia
    eos = word2idx_outputs['<eos>']

    output_sentence = []
    for _ in range(max_out_len):
        # Predicción del próximo elemento
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        idx = np.argmax(output_tokens[0, 0, :])

        # Si es "end of sentece <eos>" se acaba
        if eos == idx:
            break

        # Transformar idx a palabra
        word = ''
        if idx > 0:
            word = idx2word_target[idx]
            output_sentence.append(word)

        # Actualizar los estados dada la última predicción
        states_value = [h, c]

        # Actualizar secuencia de entrada con la salida (re-alimentación)
        target_seq[0, 0] = idx

    return ' '.join(output_sentence)

In [ ]:
i = np.random.choice(len(input_sentences))
input_seq = encoder_input_sequences[i:i+1]
translation = translate_sentence(input_seq)
print('-')
print('Input:', input_sentences[i])
print('Response:', translation)

In [ ]:
input_test = "Goodnight."
print('Input:', input_test)
integer_seq_test = input_tokenizer.texts_to_sequences([input_test])[0]
print("Representacion en vector de tokens de ids", integer_seq_test)
encoder_sequence_test = pad_sequences([integer_seq_test], maxlen=max_input_len)
print("Padding del vector:", encoder_sequence_test)

print('Input:', input_test)
translation = translate_sentence(encoder_sequence_test)
print('Response:', translation)

### 6 - Conclusión
A primera vista parece que el modelo tendría que funcionar muy bien por el accuracy alcanzado. La realidad es que las respuestas no tienen que ver demasiado con la pregunta/traducción pero la respuesta en si tiene bastante coherencia.\
Para poder mejorar el modelo haría falta poder consumir todo el dataset y todo el vocabulario, pero la cantidad de RAM no es suficiente.\
Este problema se resuelve con:
- Utilizando un DataGenerator para no levantar todo el dataset junto en el entrenamiento.
- Transfer learning evitando tener que entrenar todo el modelo  